## Step1: initialization, import necessary packages
- files are mounted from local file system "volgenmodel-docker/data" into the docker container path "/home/data" (see "docker-compose.yml")

In [1]:
import os
import sys
sys.path.append('/home/data')
from os.path import join
from functions import *
import tifffile
import SimpleITK as sitk
import numpy as np
import tifffile
import nibabel as nib
import numpy as np

os.getcwd()
os.chdir("/volgenmodel-nipype")
print(f"I'm working in folder: {os.getcwd()}")

## folder initialization
input_folder   = '/home/data/masked_brains'
output_folder  = '/home/data/output_templates'
process_folder = '/home/data/converted_brains'
tmp_folder     = '/home/data/tmp'
result_folder  = '/volgenmodel-nipype'

## parameter init
parameters={'fit_stages': 'lin,1,2', ## for testing 'lin,1,2' for best quality use larger numbers like 'lin,1,1,2,2,3,3,4,5,6' from https://doi.org/10.1016/j.ymeth.2015.01.005
            'ncpus': 8, ## how many cores should the template creation use? be careful, don't use to many or the system could crash
            'img_thresholds': len(fileList(input_folder))*[10], ## the threshold value should be the smallest grey value that is located at one of the edges of the image. 
            'padding_size': 25 ## Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)
                  }

I'm working in folder: /volgenmodel-nipype


## Step2: N4 (identical to N3) bias field correction 
- https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html


In [2]:
files = fileList(input_folder)
files = [join(input_folder,file) for file in files]
out_path = input_folder

for mri_file in files:
    create_N4_img(mri_file, mri_file.split("/")[-1] + '_N4.nii')

N4 bias correction runs.
Finished N4 Bias Field Correction.....
N4 bias correction runs.
Finished N4 Bias Field Correction.....
N4 bias correction runs.
Finished N4 Bias Field Correction.....
N4 bias correction runs.
Finished N4 Bias Field Correction.....


## Step 3: cropping based on defined threshold or auto-cropping (if available)
- to find a proper threshold load the image into imageJ and find the lowest grey-value on the border of the tissue (the croping will be done in a cubic shape from the outside)
- you can try to let the function find the threshold by themself by setting: "thresholds='auto'" (TODO: test the auto threshold function!)

In [4]:
"""
create_cropped_imgs(input_folder, 
                   input_folder, 
                   thresholds='auto', 
                   save_as_dtype=np.int16)
"""
create_cropped_imgs(input_folder, 
                   input_folder, 
                   thresholds=parameters['img_thresholds'], 
                   save_as_dtype=np.int16)


Processing image: 0120139_T1_stripped_N4_32bit.nii
Initial Dims: (250, 250, 120, 1)
Initial bestBorders: [124, 126, 59, 61]
Final bestBorders: [16, 235, 13, 102]
Number of slices after: 150
Final Dims: (150, 220, 90)
I saved to /home/data/masked_brains/0120139_T1_stripped_N4_32bit_auto-crop.nii
Processing image: 072949_T1_stripped_N4_32bit.nii
Initial Dims: (250, 250, 120, 1)
Initial bestBorders: [124, 126, 59, 61]
Final bestBorders: [16, 247, 9, 89]
Number of slices after: 148
Final Dims: (148, 232, 81)
I saved to /home/data/masked_brains/072949_T1_stripped_N4_32bit_auto-crop.nii


0

## Step 4: Zero-Padding
- Padding-Size should be ~10% of the largest dimension (88x150x249 -> paddingSize = 25)

In [5]:
create_padding_imgs(input_folder, init_paddingSize=parameters['img_thresholds'])
#create_padding_imgs(input_folder, init_paddingSize='10%')

0120139_T1_stripped_N4_32bit.nii
I padded file: 0120139_T1_stripped_N4_32bit.nii size of 26 in all 3D!
0120139_T1_stripped_N4_32bit_auto-crop.nii
I padded file: 0120139_T1_stripped_N4_32bit_auto-crop.nii size of 23 in all 3D!
072949_T1_stripped_N4_32bit.nii
I padded file: 072949_T1_stripped_N4_32bit.nii size of 26 in all 3D!
072949_T1_stripped_N4_32bit_auto-crop.nii
I padded file: 072949_T1_stripped_N4_32bit_auto-crop.nii size of 24 in all 3D!


## Step 5: Nifti to Minc convert
- it also delete possible unused mnc files in the processed folder to prevent issues

In [6]:
nii_to_minc(input_folder, process_folder)
delete_files(process_folder, 'mnc') ## delete unused .nii files in processed folder

nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse0 /home/data/converted_brains/mouse0.mnc
Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -clobber /home/data/converted_brains/mouse1 /home/data/converted_brains/mouse1.mnc


<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse0.nii'
  image_filename = '/home/data/converted_brains/mouse0.nii'
  image_offset = '352'
  ndim = '3'
  nx = '196'
  ny = '266'
  nz = '136'
  dx = '1'
  dy = '1'
  dz = '1'
  datatype = '16'
  datatype_name = 'FLOAT32'
  nvox = '7090496'
  nbyper = '4'
  byteorder = 'LSB_FIRST'
  scl_slope = '1'
  scl_inter = '0'
  sform_code = '1'
  sform_code_name = 'Scanner Anat'
  sto_xyz_matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sto_ijk matrix = '1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1'
  sform_i_orientation = 'Left-to-Right'
  sform_j_orientation = 'Posterior-to-Anterior'
  sform_k_orientation = 'Inferior-to-Superior'
  num_ext = '0'
/>
<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/home/data/converted_brains/mouse1.nii'
  image_filename = '/home/data/converted_brains/mouse1.nii'
  image_offset = '352'
  ndim = '3'
  nx = '196'
  ny = '280'
  nz = '129'
  dx = '1'
  dy = '1'
  dz = '1'
  dataty

Using s-form transform:
  1.0000,   0.0000,   0.0000,   0.0000, 
  0.0000,   1.0000,   0.0000,   0.0000, 
  0.0000,   0.0000,   1.0000,   0.0000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |     0.0000 |  1.0000 |  0.0000  0.0000  1.0000
yspace |     0.0000 |  1.0000 |  0.0000  1.0000  0.0000
xspace |     0.0000 |  1.0000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.


Copying chunks:........................................................................................................................................Done.
Copying chunks:.

mouse0.mnc -> reworked_mouse_mouse0.mnc


..............................................................................................................................

mouse1.mnc -> reworked_mouse_mouse1.mnc


..Done.


## Step 6: Create the Templates
- set up the create_parameters if necessary
- use only a small version of the fit_stages (lin,1,2) to prevent exessive calculation time
- for the best version of the template you should use the parameterset: "lin,1,2,3,4,5,6,7,8"

In [3]:
delete_files(tmp_folder, 'mnc') ## delete unused .nii files in processed folder
delete_files(tmp_folder, 'nii') ## delete unused .nii files in processed folder
process_files(process_folder, output_folder, tmp_folder, parameters, result_folder)

['/home/data/converted_brains/mouse0.mnc', '/home/data/converted_brains/mouse1.mnc']
The last modified subfolder is: /volgenmodel-nipype/workflow_temp_workflowMultiProc8
Last modification time: 2025-05-14 13:36:23
found zspace at 0 0
found yspace at 1 1
found xspace at 2 2
0 2 0 1 146 1.000000
1 1 1 1 276 1.000000
2 0 2 1 206 1.000000
TPL_mouse0_mouse1.nii
TPL_mouse0_mouse1.nii
0


0

## Step X: Remove the workflow temp or output files to restart the calculation from scratch or to save space
- will not delete the files copied to the volume outside the docker container!

In [91]:
delete_workflow_temp(search_folder="/volgenmodel-nipype/")
os.system('rm /volgenmodel-nipype/workflow_output_workflowMultiProc50/ -rf')

0

## Additional Test, check out the format of the example files from: "volgenmodel-fast-example" 
- from: https://github.com/carlohamalainen/volgenmodel-fast-example
- to load them with Fiji you have to convert em to Nifti

In [ ]:
##artifical test with test_examples (so they have to convert to nii to do the whole pipeline)
"""
testlist = os.listdir(input_folder)
for t in testlist:
    it = join(input_folder,t)
    cmd = " ".join(["mnc2nii",it, join(input_folder,t.replace("mnc","nii"))])
    os.system(cmd)
"""